In [22]:
import pandas as pd
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import yfinance as yf
from pandas import *

"""
We need to:
    - Calculate SMA over 50 -> 100 -> 200 day periods
    - Compile that data into a CSV
    - Calculate RS% over same periods (Also: April, May, June)
    - Do not include market bottom (Apr 17)
    - Stocks: TXN, AMAT, COUR, GNRC, IDCC, LLY, NICE, NOK, WFC, REGN, AMD, GRAL, KLAC
    - Compare to ^GSPC

"""
# Periods of time
time_periods = {
    'Q2_2025': ('2025-03-31', '2025-06-11'),
    'April': ('2025-04-01', '2025-04-30'),
    'May': ('2025-05-01', '2025-05-31'),
    'June': ('2025-06-01', '2025-06-11')
}

# Stocks
stock_tickers = ['TXN', 'AMAT', 'COUR', 'GNRC', 'IDCC', 'LLY', 'NICE', 'NOK', 'WFC', 'REGN', 'AMD', 'GRAL', 'KLAC']

# Pull AlphaVantageAPI
ts = TimeSeries(key="00WPSKG93DBMYOOR")
ti = TechIndicators(key='00WPSKG93DBMYOOR', output_format='pandas')

# Initialize 50, 100, 200 data for stocks (yFinance)
def initialize_sma_data(tickers, time_period, periods=[50,100,200]):
    sma_data = {}
    start, end = time_period
    for ticker in tickers:
            stock = yf.Ticker(ticker)
            data = stock.history(period="1y") # 2y historical data, enough for 200day

            for period in periods:
                sma_key = f'{ticker}_{period}_sma' # Looks like ex: TXN_50_sma
                sma = data['Close'].rolling(window=period).mean()  # Calculates SMA

                # Filter by date
                sma_data[sma_key] = sma.loc[start:end]

    return sma_data

# Insert data into separate variables based on time periods
sma_data_q2 = initialize_sma_data(stock_tickers, time_periods['Q2_2025'])
sma_data_april = initialize_sma_data(stock_tickers, time_periods['April'])
sma_data_may = initialize_sma_data(stock_tickers, time_periods['May'])
sma_data_june = initialize_sma_data(stock_tickers, time_periods['June'])

# Then into dataframes
q2_df = pd.DataFrame(sma_data_q2)
april_df = pd.DataFrame(sma_data_april)
may_df = pd.DataFrame(sma_data_may)
june_df = pd.DataFrame(sma_data_june)

# And CSVs
q2_df.to_csv('q2_sma_df.csv')
april_df.to_csv('april_sma_df.csv')
may_df.to_csv('may_sma_df.csv')
june_df.to_csv('june_sma_df.csv')


